In [214]:
using Plots
using LinearAlgebra

In [215]:
# velocidad en x, velocidad en y, tamaño de la caja, numéro de partículas.
function Átomo(vx,vy,box,N)
    ListaM = zeros(N,1)
    ListaVx = zeros(N,1)
    ListaVy = zeros(N,1)
    ListaPosx = zeros(N,1)
    ListaPosy = zeros(N,1)
    for i in 1:N
        ListaVx[i]= vx
        ListaVy[i]= vy
        ListaPosx[i] = box*rand()
        ListaPosy[i] = box*rand()
    end 
    return ListaVx, ListaVy, ListaPosx, ListaPosy
end

Átomo (generic function with 1 method)

In [216]:
#Esta función da 4 listas, que corresponden a las velocidades y posiciones (en X y Y) iniciales!
LVx, LVy, LPosx, LPosy  = Átomo(0,0,30,5)

([0.0; 0.0; … ; 0.0; 0.0], [0.0; 0.0; … ; 0.0; 0.0], [28.9673; 1.48824; … ; 14.0261; 3.33677], [17.2557; 8.23468; … ; 29.692; 12.734])

In [217]:
#Toma como argumentos LISTAS
function ForceTotal(Posx,Posy)
    dim = length(Posx)
    g = 9.81
    ϵ = 1
    σ = 1
    m = 0.018
#=Para una partícla vamos a calcular todas las fuerzas que actuan sobre ellas. Prealocamos dos listas que guarden la fuerza total 
en X y Y para cada partícula.=#
    Fuerzax = zeros(dim)
    Fuerzay = zeros(dim)
    for i in 1:dim
#Va tomando la componente y de cada partícula.
        Compy = Posy[i]
#Para ir sumando la fuerza de cada partícula "k" sobre una fija "i"
        Sumfuerzax = 0
        Sumfuerzay = 0
        for k in 1:dim
            if i != k
                Difx = Posx[i] - Posx[k]
                Dify = Posy[i] - Posy[k]
#Esta es la norma de la particula "i" con la "k"... solo para una.
                Normi = norm([Difx, Dify]) 
#Obtenemos la fuerza de Lennard
                F1 = 2.0*(ϵ/Normi)^(12)
                F2 = (ϵ/Normi)^(6)
                F3 = (24.0*σ)/((Normi)^2)
#Dos componentes de la fuerza para el 
                fuerzaix = F3*(F1 - F2)*Difx
                fuerzaiy = F3*(F1 - F2)*Dify
#Como queremos obtener la suma de las fuerzas totales sobre la partícula "i", entonces:
                Sumfuerzax = Sumfuerzax + fuerzaix
                Sumfuerzay = Sumfuerzay + fuerzaiy
            end
        end
#No se olvide que hay un factor adicional en la componente de la fuerza en y.
        Fuerzax[i] = Sumfuerzax
        Fuerzay[i] = Sumfuerzay - m*g*Compy
    end
return Fuerzax, Fuerzay
end

ForceTotal (generic function with 1 method)

In [218]:
#Aquí defino mis primeras listas de fuerzas en X y Y.
Fxinic, Fyinic, = ForceTotal(LPosx, LPosy)

([-1.79668e-7, 0.000141544, 1.1582e-7, 6.90087e-9, -0.000141487], [-3.04701, -1.45373, -1.05627, -5.24301, -2.24892])

In [219]:
#Función velocidad Verlet, debido a que ya separamos las coordenadas en posición, velocidad y fuerza, nuestra función verlet toma más argumentos.
function SpeedVerlet(Rx_0,Ry_0,Vx_0,Vy_0,Fx_0,Fy_0,h,t0,tf)
#Definiciones previas:
    arrt = t0:h:tf
    M = length(arrt)
    N = length(Rx_0) #corresponde al número de partículas
    Fac = (h^(2))/2
    L = 30 #Longitud de la caja
#Prealocamos listas:
    Forcex = zeros(M,N)
    Forcey = zeros(M,N)
    Positx = zeros(M,N)
    Posity = zeros(M,N)
    Velcyx = zeros(M,N)
    Velcyy = zeros(M,N)
#Asignamos los elementos iniciales a las matrices:
    Forcex[1,:] = Fx_0
    Forcey[1,:] = Fy_0
    Velcyx[1,:] = Vx_0
    Velcyy[1,:] = Vy_0
    Positx[1,:] = Rx_0
    Posity[1,:] = Ry_0
############ Método de velocity Verlet
   for i in 1:(M-1)
        Positx[i+1,:] = Positx[i,:] + h*Velcyx[i,:] + Fac*Forcex[i,:]
        Posity[i+1,:] = Posity[i,:] + h*Velcyy[i,:] + Fac*Forcey[i,:]
#Recordemos: a mi función ForceTotal le doy dos listas y me regresa dos listas, Fuerzax y Fuerzay
        F_nextx, F_nexty = ForceTotal(Positx[i+1,:],Posity[i+1,:])
        Forcex[i+1,:] = F_nextx
        Forcey[i+1,:] = F_nexty
#Hasta aquí ya calculó la fuerza F_k+1, entonces ya puede encontrar velocidad:
        Velcyx[i+1,:] = Velcyx[i,:] + (h*0.5)*(Forcex[i,:] + Forcex[i+1,:])
        Velcyy[i+1,:] = Velcyy[i,:] + (h*0.5)*(Forcey[i,:] + Forcey[i+1,:])
    end
    #return Forcex, Forcey,Positx,Posity,Velcyx,Velcyy
    return Posity
end

SpeedVerlet (generic function with 1 method)

In [220]:
SpeedVerlet(LPosx,LPosy,LVx,LVy,Fxinic,Fyinic,0.25,0,4)

17×5 Array{Float64,2}:
 17.2557     8.23468     5.9818     29.692     12.734   
 17.1605     8.18925     5.9488     29.5281    12.6637  
 16.8759     8.05346     5.85013    29.0384    12.4537  
 16.405      7.82882     5.68691    28.2282    12.1062  
 15.7531     7.51781     5.46092    27.1065    11.625   
 14.9274     7.12386     5.17467    25.6856    11.0155  
 13.9369     6.65135     4.83131    23.9812    10.2844  
 12.7925     6.1055      4.43462    22.0122     9.4397  
 11.5071     5.49238     3.989      19.8003     8.49072 
 10.0946     4.81883     3.49935    17.3698     7.44784 
  8.57067    4.09245     2.97108    14.7476     6.32243 
  6.95219    3.32158     2.41003    11.9627     5.12656 
  5.25698    2.51542     1.82237     9.04573    3.87274 
  3.50376    1.68419     1.21461     6.02894    2.57349 
  1.71186    0.838854    0.593435    2.94562    1.24137 
 -0.098924  -0.0113925  -0.0342844  -0.170212  -0.108811
 -1.90862   -0.863205   -0.661626   -3.28417   -1.45609 

In [221]:
#Función velocidad Verlet, debido a que ya separamos las coordenadas en posición, velocidad y fuerza, nuestra función verlet toma más argumentos.
function SpeedVerlet2(Rx_0,Ry_0,Vx_0,Vy_0,Fx_0,Fy_0,h,t0,tf)
#Definiciones previas:
    arrt = t0:h:tf
    M = length(arrt)
    N = length(Rx_0) #corresponde al número de partículas
    Fac = (h^(2))/2
    L = 30 #Longitud de la caja
    Bd = 0
    Bu = L
    Bl = 0
    Br = L
#Prealocamos listas:
    Forcex = zeros(M,N)
    Forcey = zeros(M,N)
    Positx = zeros(M,N)
    Posity = zeros(M,N)
    Velcyx = zeros(M,N)
    Velcyy = zeros(M,N)
#Asignamos los elementos iniciales a las matrices:
    Forcex[1,:] = Fx_0
    Forcey[1,:] = Fy_0
    Velcyx[1,:] = Vx_0
    Velcyy[1,:] = Vy_0
    Positx[1,:] = Rx_0
    Posity[1,:] = Ry_0
############ Método de velocity Verlet
for i in 1:(M-1)
        Positx[i+1,:] = Positx[i,:] + h*Velcyx[i,:] + Fac*Forcex[i,:]
        Posity[i+1,:] = Posity[i,:] + h*Velcyy[i,:] + Fac*Forcey[i,:]
#Vamos a revisar que las posiciones no hayan atravesado la caja
    for k in 1:N
        if (Positx[i+1,k] < Bl)
            Positx[i+1,k] = Bl - (Positx[i+1,k] - Bl)
            Velcyx[i+1,k] = (-1)*Velcyx[i+1,k]
        end
        if (Positx[i+1,k] > Br)
            Positx[i+1,k] = Br - (Positx[i+1,k] - Br)
            Velcyx[i+1,k] = (-1)*Velcyx[i+1,k]
        end
        if (Posity[i+1,k] < Bd)
            Posity[i+1,k] = Bd - (Posity[i+1,k] - Bd)
            Velcyy[i,k] =  (-1)*Velcyy[i,k]
        end
        if (Posity[i+1,k] > Bu)
            Posity[i+1,k] = Bu - (Posity[i+1,k] - Bu)
            Velcyy[i,k] = (-1)*Velcyy[i,k]
        end
    end
#Recordemos: a mi función ForceTotal le doy dos listas y me regresa dos listas, Fuerzax y Fuerzay
        F_nextx, F_nexty = ForceTotal(Positx[i+1,:],Posity[i+1,:])
        Forcex[i+1,:] = F_nextx
        Forcey[i+1,:] = F_nexty
#Hasta aquí ya calculó la fuerza F_k+1, entonces ya puede encontrar velocidad:
        Velcyx[i+1,:] = Velcyx[i,:] + (h*0.5)*(Forcex[i,:] + Forcex[i+1,:])
        Velcyy[i+1,:] = Velcyy[i,:] + (h*0.5)*(Forcey[i,:] + Forcey[i+1,:])
    end
    #return Forcex, Forcey,Positx,Posity,Velcyx,Velcyy
    return Velcyy
end

SpeedVerlet2 (generic function with 1 method)

In [222]:
SpeedVerlet2(LPosx,LPosy,LVx,LVy,Fxinic,Fyinic,0.25,0,4)

17×5 Array{Float64,2}:
  0.0        0.0        0.0         0.0       0.0    
 -0.759651  -0.36243   -0.263338   -1.30714  -0.56068
 -1.51092   -0.720853  -0.52377    -2.59985  -1.11518
 -2.24551   -1.07131   -0.778421   -3.86386  -1.65738
 -2.95532   -1.40992   -1.02448    -5.08524  -2.18132
 -3.63252   -1.73292   -1.25924    -6.25049  -2.68122
 -4.26962   -2.03674   -1.48009    -7.34676  -3.1516 
 -4.85961   -2.31794   -1.68461    -8.36195  -3.58733
 -5.39596   -2.57333   -1.87054    -9.28486  -3.9837 
 -5.87276   -2.79986   -2.03583   -10.1053   -4.33658
 -6.28475   -2.9945    -2.17865   -10.8142   -4.64257
 -6.62738   -3.15406   -2.29742   -11.4038   -4.89937
 -6.89687   -3.27476   -2.39084   -11.8675   -5.10614
 -7.09024   -3.35313   -2.45787   -12.2002   -5.26275
  7.20536    3.39117    2.49778    12.3983    5.3646 
  7.16539    3.3845     2.48393    12.3295    5.32271
  7.1215     3.39168    2.46871    12.254     5.26219